In [3]:
import numpy as np
from scipy.stats import norm

First, a bit on notation.  We will denote the distribution functions as follows:<br>
$\phi = $ Normal probability density function <br>
$\Phi = $ normal cumulative density function

For more information on the normal distribution see https://en.wikipedia.org/wiki/Normal_distribution

We define the following symbols:
- $S$ = Stock Price<br>
- $K$ = Strike price<br>
- $t$ = Time to expiration (years)<br>
- $r$ = Risk-free rate<br>
- $\sigma$ = Implied Volatility

The Black Scholes model gives:
- Call Price = $S\Phi(d_1) -Ke^{-rt}\Phi(d_2)$ <br>
- Put Price = $-S\Phi(-d_1) + Ke^{-rt}\Phi(-d_2)$

In the above equations, $d_1$ and $d_2$ are given respectively by,
$$d_1 = \frac{1}{\sigma \sqrt{t}} \left[ \ln\left(\frac{S}{K}\right) + \left(r + \frac{\sigma.^2}{2}\right) t\right],$$
and
$$d_2 = d_1 - \sigma \sqrt{t}.$$

Functions that return the Black-Scholes price of an option and the values of $d_1$ and $d_2$ are given in the cell below.

In [4]:
#  Functions that return d_1, d_2 and call and put prices
def d(sigma, S, K, r, t):
    d1 = 1 / (sigma * np.sqrt(t)) * ( np.log(S/K) + (r + sigma**2/2) * t)
    d2 = d1 - sigma * np.sqrt(t)
    return d1, d2

def call_price(sigma, S, K, r, t, d1, d2):
    C = norm.cdf(d1) * S - norm.cdf(d2) * K * np.exp(-r * t)
    return C

def put_price(sigma, S, K, r, t, d1, d2):
    P = -norm.cdf(-d1) * S + norm.cdf(-d2) * K * np.exp(-r * t)
    return P

From the Wikipedia article, we get the expressions for $\Delta$, $\Gamma$, and $\Theta$:

$$\Delta_{\mbox{call}} = \Phi(d_1)$$
$$\Delta_{\mbox{put}} = \Phi(-d_1)$$

Gamma is the same for both calls and puts,
$$\Gamma = Ke^{-et} \frac{\phi(d_2)}{S^2\sigma\sqrt{t}} $$

For $\Theta$ we have,
$$\Theta_{\mbox{call}} = -\frac{S\sigma\phi(d_1)}{2\sqrt{t}} -rKe^{-rt}\Phi(d_2)$$
$$\Theta_{\mbox{put}} = -\frac{S\sigma\phi(-d_1)}{2\sqrt{t}} -rKe^{-rt}\Phi(-d_2)$$

In [21]:
#  Functions for Deltam Gamma, and  Theta
def delta(d_1, contract_type):
    if contract_type == 'c':
        return norm.cdf(d1)
    if contract_type == 'p':
        return -norm.cdf(-d_1)
    
def gamma(d2, S, K, sigma, r, t):
    return( K * np.exp(-r * t) * (norm.pdf(d2) / (S**2 * sigma * np.sqrt(t) ))) 

def theta(d1, d2, S, K, sigma, r, t, contract_type):
    if contract_type == 'c':
        theta = -S * sigma * norm.pdf(d1) / (2 * np.sqrt(t)) - r * K * np.exp(-r * t) * norm.cdf(d2)
    if contract_type == 'p':
        theta = -S * sigma * norm.pdf(-d1) / (2 * np.sqrt(t)) + r * K * np.exp(-r * t) * norm.cdf(-d2)

    return theta

def vega(sigma, S, K, r, t):
    d1, d2 = d(sigma, S, K, r, t)
    v = S * norm.pdf(d1) * np.sqrt(t)
    return v

In [22]:
#  Using TSLA data from 5/9/2020 closing values
S = 819.42; print('S = ', S)
K = 1020; print('K = ', K)
r = 0.01; print('r = ', r)
t = 42 / 365; print('t = ', t)
sigma = 0.6966; print('sigma = ', sigma)

S =  819.42
K =  1020
r =  0.01
t =  0.11506849315068493
sigma =  0.6966


In [23]:
#  Calculate the values of d1 and d2 needed for other functions
d1, d2 = d(sigma, S, K, r, t)
print('d1 = ', d1)
print('d2 = ', d2)

d1 =  -0.8036087890894198
d2 =  -1.0399076799209663


In [24]:
delta_call = delta(d1, 'c')
delta_put = delta(d1, 'p')
print('Call Delta = ', delta_call)
print('Put Delta = ', delta_put)

Call Delta =  0.21081147278433515
Put Delta =  -0.7891885272156649


In [25]:
print( 'Gamma = ', gamma(d2, S, K, sigma, r, t) )

Gamma =  0.0014918011547566806


In [26]:
#####  Calculate Theta
print(S, K, r, t, sigma, d1, d2)
print( theta(d1, d2, S, K, sigma, r, t, 'c') )

print( 'Call Theta = ', theta(d1, d2, S, K, sigma, r, t, 'c') / 365 * 100)
print( 'Put Thata = ', theta(d1, d2, S, K, sigma, r, t, 'p') / 365 * 100)

819.42 1020 0.01 0.11506849315068493 0.6966 -0.8036087890894198 -1.0399076799209663
-244.55062685827485
Call Theta =  -67.00017174199311
Put Thata =  -64.2088649573641


#  Discussion of Units
Option price $\rightarrow$ price per share<br>
S $\rightarrow$  price per share <br>
K $\rightarrow$  price per share <br>
r $\rightarrow$  per year<br>
t $\rightarrow$  Years<br>
$\sigma \rightarrow$  per $\sqrt{\mbox{year}}$

One contract = 100 shares

## Example with $\Theta$
Fromthe Black Scholes model we see that $\Theta$ has units of price per share per year,
$$\Theta == \frac{\partial V}{\partial t}\rightarrow \left( \frac{\mbox{price}}{\mbox{share}}\right)\left(\frac{1}{\mbox{years}}\right)$$

But the convention for most trading platforms is,
$$\left(\frac{\mbox{price}}{\mbox{contract}}\right)\left(\frac{1}{\mbox{day}}\right)$$

We convert to price per contract per day by coming up with a conversion factor.
$$\left( \frac{\mbox{price}}{\mbox{share}}\right)\left(\frac{1}{\mbox{years}}\right) \times
\left(\frac{100\mbox{shares}}{1\mbox{contract}}\right)\left(\frac{1\mbox{year}}{365\mbox{days}}\right)$$

Using AAPL as an example,
<IMG SRC="aapl_theta.png" WIDTH="40%">

In [27]:
S = 120.96
K = 135
r = 0.01
t = 14 / 365
sigma = 0.6427

In [28]:
d1, d2 =  d(sigma, S, K, r, t)
print( theta(d1, d2, S, K, sigma, r, t, 'c') * 100 / 365) 

-15.73578917740298


## Example with vega ($\nu$)

Bega has units of,
$$  \mbox{vega} = \frac{\partial V}{\partial \sigma} \rightarrow \left(\frac{\mbox{price}}{\mbox{share}}\right)\left(\frac{1}{\mbox{vol point}}\right)$$

The platform quotes vega in units of price per share per vol percentage point.  To do the conversion we do the following,
$$  \mbox{vega} = \frac{\partial V}{\partial \sigma} \rightarrow \left(\frac{\mbox{price}}{\mbox{share}}\right)\left(\frac{1}{\mbox{vol point}}\right)
\left(\frac{1 \mbox{vol point}}{100\mbox{percentage points}}\right)$$
<IMG SRC="aapl_vega.png" WIDTH="40%">

In [29]:
print( vega(sigma, S, K, r, t) /100 )

0.06827204317718513
